In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.metrics import Precision, Recall


In [6]:
# Load Nutrient Data
csv_path = 'dataset/indian_dishes_nutrients.csv'  # Ensure columns: 'Food', 'Calories', 'Protein', 'Fat', 'Carbs'
data = pd.read_csv(csv_path)
X_nutrients = data.drop(columns=['Food'])
y = data['Food']

In [7]:
# Encode Food Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [9]:
# Load Image Data
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
image_size = (128, 128)
batch_size = 32

train_images = data_gen.flow_from_directory(
    'dataset/Indian Food Images',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'
)

val_images = data_gen.flow_from_directory(
    'dataset/Indian Food Images',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'
)

Found 3200 images belonging to 80 classes.
Found 800 images belonging to 80 classes.


In [10]:
train_images.class_indices

{'adhirasam': 0,
 'aloo_gobi': 1,
 'aloo_matar': 2,
 'aloo_methi': 3,
 'aloo_shimla_mirch': 4,
 'aloo_tikki': 5,
 'anarsa': 6,
 'ariselu': 7,
 'bandar_laddu': 8,
 'basundi': 9,
 'bhatura': 10,
 'bhindi_masala': 11,
 'biryani': 12,
 'boondi': 13,
 'butter_chicken': 14,
 'chak_hao_kheer': 15,
 'cham_cham': 16,
 'chana_masala': 17,
 'chapati': 18,
 'chhena_kheeri': 19,
 'chicken_razala': 20,
 'chicken_tikka': 21,
 'chicken_tikka_masala': 22,
 'chikki': 23,
 'daal_baati_churma': 24,
 'daal_puri': 25,
 'dal_makhani': 26,
 'dal_tadka': 27,
 'dharwad_pedha': 28,
 'doodhpak': 29,
 'double_ka_meetha': 30,
 'dum_aloo': 31,
 'gajar_ka_halwa': 32,
 'gavvalu': 33,
 'ghevar': 34,
 'gulab_jamun': 35,
 'imarti': 36,
 'jalebi': 37,
 'kachori': 38,
 'kadai_paneer': 39,
 'kadhi_pakoda': 40,
 'kajjikaya': 41,
 'kakinada_khaja': 42,
 'kalakand': 43,
 'karela_bharta': 44,
 'kofta': 45,
 'kuzhi_paniyaram': 46,
 'lassi': 47,
 'ledikeni': 48,
 'litti_chokha': 49,
 'lyangcha': 50,
 'maach_jhol': 51,
 'makki_di_

In [11]:
class_names = list(train_images.class_indices.keys())
class_names

['adhirasam',
 'aloo_gobi',
 'aloo_matar',
 'aloo_methi',
 'aloo_shimla_mirch',
 'aloo_tikki',
 'anarsa',
 'ariselu',
 'bandar_laddu',
 'basundi',
 'bhatura',
 'bhindi_masala',
 'biryani',
 'boondi',
 'butter_chicken',
 'chak_hao_kheer',
 'cham_cham',
 'chana_masala',
 'chapati',
 'chhena_kheeri',
 'chicken_razala',
 'chicken_tikka',
 'chicken_tikka_masala',
 'chikki',
 'daal_baati_churma',
 'daal_puri',
 'dal_makhani',
 'dal_tadka',
 'dharwad_pedha',
 'doodhpak',
 'double_ka_meetha',
 'dum_aloo',
 'gajar_ka_halwa',
 'gavvalu',
 'ghevar',
 'gulab_jamun',
 'imarti',
 'jalebi',
 'kachori',
 'kadai_paneer',
 'kadhi_pakoda',
 'kajjikaya',
 'kakinada_khaja',
 'kalakand',
 'karela_bharta',
 'kofta',
 'kuzhi_paniyaram',
 'lassi',
 'ledikeni',
 'litti_chokha',
 'lyangcha',
 'maach_jhol',
 'makki_di_roti_sarson_da_saag',
 'malapua',
 'misi_roti',
 'misti_doi',
 'modak',
 'mysore_pak',
 'naan',
 'navrattan_korma',
 'palak_paneer',
 'paneer_butter_masala',
 'phirni',
 'pithe',
 'poha',
 'poornalu

In [12]:
# Image Model
image_input = Input(shape=(128, 128, 3))
x = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
image_output = layers.Dense(128, activation='relu')(x)

In [13]:
# Classification Layer
final_output = layers.Dense(len(label_encoder.classes_), activation='softmax')(image_output)

In [14]:
# Define the Model
model = models.Model(inputs=image_input, outputs=final_output)

In [25]:
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 80)             │        10,320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,207,922 (84.72 MB)

 Trainable params: 7,402,640 (28.24 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 14,805,282 (56.48 MB)

In [17]:
# Train the Model
epochs = 50
model.fit(
    train_images,
    epochs=epochs,
    validation_data=val_images
)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


2025-02-20 11:30:36.058148: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,2560] vs. [1,32]
	 [[{{function_node __inference_one_step_on_data_2038}}{{node LogicalAnd}}]]


InvalidArgumentError: Graph execution error:

Detected at node LogicalAnd defined at (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code

  File "/Users/aditichavan/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/aditichavan/Library/Python/3.10/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/aditichavan/Library/Python/3.10/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/aditichavan/Library/Python/3.10/lib/python/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/Library/Frameworks/Python.framework

In [18]:
# Compile the Model with additional metrics like Precision and Recall
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy', Precision(), Recall()])

In [19]:
history = model.fit(train_images, epochs=10, validation_data=val_images)

# Accessing the history object for training metrics
print(f"Training Accuracy: {history.history['accuracy'][-1]}")
print(f"Validation Accuracy: {history.history['val_accuracy'][-1]}")
print(f"Training Precision: {history.history['precision'][-1]}")
print(f"Validation Precision: {history.history['val_precision'][-1]}")
print(f"Training Recall: {history.history['recall'][-1]}")
print(f"Validation Recall: {history.history['val_recall'][-1]}")


Epoch 1/10


2025-02-20 11:31:01.381018: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,2560] vs. [1,32]
	 [[{{function_node __inference_one_step_on_data_4063}}{{node LogicalAnd}}]]


InvalidArgumentError: Graph execution error:

Detected at node LogicalAnd defined at (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code

  File "/Users/aditichavan/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/aditichavan/Library/Python/3.10/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/aditichavan/Library/Python/3.10/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/aditichavan/Library/Python/3.10/lib/python/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/Library/Frameworks/Python.framework

In [20]:
import pickle

# Save the trained model
model.save('model.h5')  # Save the Keras model in HDF5 format

# Save the label encoder
with open('model.pkl', 'wb') as encoder_file:
    pickle.dump(label_encoder, encoder_file)

print("model.h5'")
print("model.pkl'")


PermissionError: [Errno 13] Unable to create file (unable to open file: name = 'model.h5', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 602)

In [21]:
# Load the saved model
from tensorflow.keras.models import load_model

loaded_model = load_model('model.h5')
print("Model loaded successfully!")

# Load the saved label encoder
with open('model.pkl', 'rb') as encoder_file:
    loaded_encoder = pickle.load(encoder_file)
print("Label Encoder loaded successfully!")


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [39]:
def predict_food_and_nutrients_with_loaded_model(image_path):
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict using the loaded model
    prediction = loaded_model.predict(img_array)
    predicted_class = loaded_encoder.inverse_transform([np.argmax(prediction)])[0]

    # Retrieve the nutrients for the predicted food
    nutrients = data[data['Food'] == predicted_class].iloc[:, 1:].to_dict('records')[0]
    return predicted_class, nutrients

# Example Usage
example_image = '33e8639efd.jpg'
predicted_food, predicted_nutrients = predict_food_and_nutrients_with_loaded_model(example_image)
print(f'Predicted Food: {predicted_food}')
print(f'Nutrients: {predicted_nutrients}')


1/1 [==============================] - 0s 59ms/step
Predicted Food: butter_chicken
Nutrients: {'Proteins (g)': 23.0, 'Carbs (g)': 16.5, ' Sugar (g)': 7.0, 'Fats (g)': 14.3}
